# Diplodatos Kaggle Competition

We present this peace of code to create the baseline for the competition, and as an example of how to deal with these kind of problems. The main goals are that you:

1. Learn
1. Try different models and see which one fits the best the given data
1. Get a higher score than the given one in the current baseline example
1. Try to get the highest score in the class :)

In [1]:
# Import the required packages
import os # manejar archivos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load the given labels
breed = pd.read_csv('../data/breed_labels.csv')
color = pd.read_csv('../data/color_labels.csv')
state = pd.read_csv('../data/state_labels.csv')

Now we take a look at the labels, just to understand what these are

In [3]:
breed.head()

,BreedID,Type,BreedName
0,1,1,Affenpinscher
1,2,1,Afghan Hound
2,3,1,Airedale Terrier
3,4,1,Akbash
4,5,1,Akita


In [4]:
# Type 1 is dog, 2 is cat
breed.shape # hay 307 razas

(307, 3)

In [5]:
color.head(7)

,ColorID,ColorName
0,1,Black
1,2,Brown
2,3,Golden
3,4,Yellow
4,5,Cream
5,6,Gray
6,7,White


In [6]:
color.shape # 7 colores

(7, 2)

In [7]:
# Nombre del estado
state 

,StateID,StateName
0,41336,Johor
1,41325,Kedah
2,41367,Kelantan
3,41401,Kuala Lumpur
4,41415,Labuan
5,41324,Melaka
6,41332,Negeri Sembilan
7,41335,Pahang
8,41330,Perak
9,41380,Perlis


In [8]:
state.shape 

(15, 2)

And now we are ready to deal with the *original* dataset...

In [9]:
original_df = pd.read_csv('../data/train.csv')

In [10]:
original_df.columns # 20 columnas. No se consideran: name, video, photo, rescueid

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'Description',
       'AdoptionSpeed', 'PID'],
      dtype='object')

In [11]:
original_df.describe() # 10582 pet en train

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,AdoptionSpeed,PID
count,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000,10582.000000
mean,1.454734,10.520412,265.469854,74.388868,1.779059,2.230675,3.236912,1.856738,1.860518,1.460971,1.729730,1.566528,1.912115,1.036666,1.584011,20.809960,41345.994613,2.518900,7477.025799
std,0.497970,18.374027,60.121490,123.434010,0.684763,1.743985,2.748595,2.974465,0.547535,0.593843,0.670791,0.701482,0.564041,0.198228,1.488348,78.397243,32.409109,1.176018,4310.921553
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41324.000000,0.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,2.000000,3768.250000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,2.000000,7473.500000
75%,2.000000,12.000000,307.000000,188.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,41401.000000,4.000000,11200.750000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,20.000000,3000.000000,41415.000000,4.000000,14992.000000


Para ver si el dataset esta desbalanceado:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

In [12]:
original_df[['AdoptionSpeed','Type']].groupby('AdoptionSpeed').agg('count')

,Type
AdoptionSpeed,
0,285
1,2177
2,2846
3,2310
4,2964


No consideramos que este muy desbalanceado como para tomar alguna medida de antemano, entiéndase como submuestreo o sobremuestreo.

In [13]:
original_df.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,Description,AdoptionSpeed,PID
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,Nibble is a 3+ month old ball of cuteness. He ...,2,0
1,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,"Good guard dog, very alert, active, obedience ...",2,3
2,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,This handsome yet cute boy is up for adoption....,2,4
3,2,3,266,0,2,5,6,0,2,1,2,2,2,1,1,0,41326,This is a stray kitten that came to my house. ...,2,5
4,2,12,264,264,1,1,0,0,2,3,2,2,3,1,1,300,41326,anyone within the area of ipoh or taiping who ...,1,6


Create a function to transform the datasets. This is done by means of a function so that the transformations are the same for the training and testing datasets... We replace the encodings just to make it easy to "visualize" the data

In [14]:
def transform_data(train_data_fname, test_data_fname):
    def transform_columns(df):
        df = df.drop(["Description"], axis=1)
        df.Type = df.Type.replace({1: 'Dog', 2: 'Cat'})
        df.Gender = df.Gender.replace({1:'Male', 2:'Female', 3:'Mixed'})
        df.MaturitySize = df.MaturitySize.replace({1:'S', 2:'M', 3:'L', 4:'XL', 0:'N/A'})
        df.FurLength = df.FurLength.replace({1:'S', 2:'M', 3:'L', 0:'N/A'})
        df.Vaccinated = df.Vaccinated.replace({1:'T', 2:'N', 3:'N/A'})
        df.Dewormed = df.Dewormed.replace({1:'T', 2:'F', 3:'N/A'})
        df.Sterilized = df.Sterilized.replace({1:'T', 2:'F', 3:'N/A'})
        df.Health = df.Health.replace({1:'Healthy', 2: 'MinorInjury', 3:'SeriousInjury', 0: 'N/A'})
        df.Color1 = df.Color1.replace(dict(list(zip(color.ColorID, color.ColorName)) + [(0, "N/A")]))
        df.Color2 = df.Color2.replace(dict(list(zip(color.ColorID, color.ColorName)) + [(0, "N/A")]))
        df.Color3 = df.Color3.replace(dict(list(zip(color.ColorID, color.ColorName)) + [(0, "N/A")]))
        df.Breed1 = df.Breed1.replace(dict(list(zip(breed.BreedID, breed.BreedName)) + [(0, "N/A")]))
        df.Breed2 = df.Breed2.replace(dict(list(zip(breed.BreedID, breed.BreedName)) + [(0, "N/A")]))
        return df
    # breed lo usa dos veces para raza pura y mezclada
    # color lo usa 3 veces para 3 colores de la mascota
       
    df_train = pd.read_csv(train_data_fname)
    df_train = transform_columns(df_train)
    df_test = pd.read_csv(test_data_fname)
    df_test = transform_columns(df_test)
    
    # SE ELIMINA LA COLUMNA DESCRIPTION Y SE REEMPLAZAN LOS ENCODINGS POR SU SIGNIFICADO.
    
    df = pd.concat([df_train, df_test], sort=True) 
    # sort: Ordena el eje de no concatenación si aún no está alineado
    # concatena para considerar todas las opciones al convertir en dummies

    # set dummy variables for everything
    # except from Age, Quantity, Fee
    df = pd.get_dummies(df) # Convertir variables categóricas en indicadores (codificación de bits 0s y 1s)
    # get train and test back. vuelve a separar
    n = len(df_train)
    df_train = df.iloc[:n]
    df_test = df.iloc[n:]
    
    y = df_train['AdoptionSpeed'] # target 0,1,2,3,4
    X = df_train.drop('AdoptionSpeed', axis=1)
    yy = None
    XX = df_test.drop('AdoptionSpeed', axis=1)

    return X, y, XX, yy

Load the data...

In [15]:
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")
# X,y train
# XX,yy test

Create the model and evaluate it

In [16]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it couldn be useful for you depending on your approach)
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

# resultados de los modelos entrenados
results = pd.DataFrame(columns=('clf', 'best_acc')) # clasificador, mejor accuracy

In [17]:
from sklearn.tree import DecisionTreeClassifier as DT


tree_param = {'criterion':('gini', 'entropy'), 'min_samples_leaf':(1, 2, 5),
              'min_samples_split':(2, 3, 5, 10, 50, 100)}
# define el modelo a usar: decision tree
tree = DT(random_state=42)
# defino la búsqueda de hiperparámetros con GridSearchCV
tree_clf = GridSearchCV(tree, tree_param, scoring='accuracy', cv=3, iid=False)
# llamo al método fit de GridSearchCV para introducir los datos de entrenamiento. Y entrena
tree_clf.fit(X_train.drop(["PID"], axis=1), y_train)
# pregunto cuál fue el mejor estimador
best_tree_clf = tree_clf.best_estimator_
print('Best Decision Tree accuracy: ', tree_clf.best_score_)
print(best_tree_clf)

# Guardo los resultados en el DataFrame creado
results = results.append({'clf': best_tree_clf, 'best_acc': tree_clf.best_score_}, ignore_index=True)

# printeo el mejor clasificador según mayor score. hasta ahora solo tengo el tree 
print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

Best Decision Tree accuracy:  0.3519622095560508
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')
The best classifier so far is: 
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')


In [18]:
results

,clf,best_acc
0,"DecisionTreeClassifier(class_weight=None, crit...",0.351962


In [19]:
X_train.columns # separó todos los posibles features en columnas

Index(['Age', 'Fee', 'PID', 'Quantity', 'State', 'Breed1_Abyssinian',
       'Breed1_Affenpinscher', 'Breed1_Airedale Terrier', 'Breed1_Akita',
       'Breed1_American Bulldog',
       ...
       'MaturitySize_S', 'MaturitySize_XL', 'Sterilized_F', 'Sterilized_N/A',
       'Sterilized_T', 'Type_Cat', 'Type_Dog', 'Vaccinated_N',
       'Vaccinated_N/A', 'Vaccinated_T'],
      dtype='object', length=360)

In [20]:
X.shape, XX.shape # quedaron 360 columnas de features

((10582, 360), (4411, 360))

**And finally**, we predict the unknown label for the testing set

In [21]:
yy = results.clf.iloc[0].predict(XX.drop(["PID"], axis=1))
yy = yy.astype(np.int) # sin np.int me queda: [2., 1., 2., ..., 2., 4., 4.]

The last thing we do is generating a file that should be *submitted* on kaggle

In [22]:
# subimos un dataframe con el pid de test y las predicciones de AdoptionSpeed: yy
submission = pd.DataFrame(data= list(zip(XX.PID, yy)), columns=["PID", "AdoptionSpeed"])
# ellos se guardan los labels del conjunto de test

In [23]:
# lo guardamos en un csv que vamos a subir a kaggle
submission.to_csv("../data/submission.csv", header=True, index=False)

## Resolución:

### Árbol de decisión con otros parámetros

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [24]:
# tree_param = {'criterion':('gini', 'entropy'), 'min_samples_leaf':(1, 2, 5),
#               'min_samples_split':(2, 3, 5, 10, 50, 100), 'max_depth':(5,10,15,20), 'class_weight':[None,'balanced'] }

tree_param = {'criterion':('gini', 'entropy'), 'min_samples_leaf':(1,2,3),
              'min_samples_split':(100, 200, 300), 'max_depth':(7,10,13)}

tree = DT(random_state=42)
tree_clf = GridSearchCV(tree, tree_param, scoring='accuracy', cv=5, iid=False)
tree_clf.fit(X.drop(["PID"], axis=1), y)
best_tree_clf = tree_clf.best_estimator_
print('Best Decision Tree accuracy: ', tree_clf.best_score_)
print(best_tree_clf)


Best Decision Tree accuracy:  0.3800769141715768
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=200,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')


In [25]:
results = results.append({'clf': best_tree_clf, 'best_acc': tree_clf.best_score_}, ignore_index=True)
results

,clf,best_acc
0,"DecisionTreeClassifier(class_weight=None, crit...",0.351962
1,"DecisionTreeClassifier(class_weight=None, crit...",0.380077


### Random forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
grid_param = {'n_estimators':(10,20,30), 'criterion':('gini', 'entropy'), 'max_depth':(7,10,13),'min_samples_leaf':(1,2,3),
              'min_samples_split':(100, 200, 300) }

RF = RandomForestClassifier(random_state=42)
RF_clf = GridSearchCV(RF, grid_param, scoring='accuracy', cv=5, iid=False)
RF_clf.fit(X.drop(["PID"], axis=1), y)
best_RF_clf = RF_clf.best_estimator_
print('Best Decision Tree accuracy: ', RF_clf.best_score_)
print(best_RF_clf)

results = results.append({'clf': best_RF_clf, 'best_acc': RF_clf.best_score_}, ignore_index=True)

Best Decision Tree accuracy:  0.3776207878484451
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=13, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [18]:
grid_param = {'n_estimators':(15,20,25), 'criterion':('gini', 'entropy'), 'max_depth':(13,16,19),'min_samples_leaf':(3,5,7),
              'min_samples_split':(50,100,150) }

RF = RandomForestClassifier(random_state=42)
RF_clf = GridSearchCV(RF, grid_param, scoring='accuracy', cv=5, iid=False)
RF_clf.fit(X.drop(["PID"], axis=1), y)
best_RF_clf = RF_clf.best_estimator_
print('Best Decision Tree accuracy: ', RF_clf.best_score_)
print(best_RF_clf)

results = results.append({'clf': best_RF_clf, 'best_acc': RF_clf.best_score_}, ignore_index=True)

Best Decision Tree accuracy:  0.38026693376014115
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=19, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=15,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [19]:
# yy= RF_clf.predict(XX.drop(["PID"], axis=1)).astype(np.int)
# submission = pd.DataFrame(list(zip(XX.PID, yy)), columns=["PID", "AdoptionSpeed"])
# submission.to_csv("../data/submission.csv", header=True, index=False)

In [30]:
grid_param = {'n_estimators':(7,10,13), 'criterion':('gini', 'entropy'), 'max_depth':(20,23),'min_samples_leaf':(3,5,7),
              'min_samples_split':(75,100,125) }

RF = RandomForestClassifier(random_state=42)
RF_clf = GridSearchCV(RF, grid_param, scoring='accuracy', cv=5, iid=False)
RF_clf.fit(X.drop(["PID"], axis=1), y)
best_RF_clf = RF_clf.best_estimator_
print('Best Decision Tree accuracy: ', RF_clf.best_score_)
print(best_RF_clf)

results = results.append({'clf': best_RF_clf, 'best_acc': RF_clf.best_score_}, ignore_index=True)

Best Decision Tree accuracy:  0.38007856603126966
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=75,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


### SGDClassifier

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [31]:
from sklearn.linear_model import SGDClassifier

In [32]:
# LOS MEAN_TEST_SCORE SON BAJOS, NO PASAN LOS 0.25, POR ESO NO VUELVO A CORRER EL BLOQUE

# plt.figure(figsize=(8,3), dpi= 80, facecolor='w', edgecolor='k')
# 
# # funcs_costo = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
# funcs_costo = ['hinge', 'log', 'perceptron']
# tiporeg = ['l2', 'l1', 'elasticnet']
# mejormeanscore = 0
# mejorloss = ''
# mejorpenalty = ''
# mejoreta0 = 0
# mejoralpha = 0
# 
# for loss in funcs_costo:
#     for penalty in tiporeg:
#         print('\n\nFunción de costo: ', loss, '\nTipo de regularizador: ', penalty, '\nScore: accuracy')
#         
#         param_grid = {'eta0': [0.1, 0.01, 0.001],  # Tasa de entrenamiento
#                     'alpha': [0.1, 0.01, 0.001]  # Tasa de regularización
#         }
#         graddes = SGDClassifier(loss=loss, tol=0.001, learning_rate='optimal', penalty=penalty, random_state=0) 
#         # learning_rate es 'optimal' por default
#         # , class_weight={0: 0.8, 1: 0.2} or "balanced" da peores resultados
# 
#         model = GridSearchCV(estimator=graddes, param_grid=param_grid, cv=5, scoring='accuracy', iid=True)    
#         # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
#         # cv= integer, to specify the number of folds in a (Stratified)KFold: The folds are made 
#         # by preserving the percentage of samples for each class.
#         model.fit(X.drop(["PID"], axis=1), y)
# 
#         columns= ['param_alpha', 'param_eta0', 'mean_test_score', 'std_test_score']
#         resultados = np.c_[ np.array(model.cv_results_['param_alpha']), np.array(model.cv_results_['param_eta0']), 
#                            np.array(model.cv_results_['mean_test_score']), np.array(model.cv_results_['std_test_score'])]
#         resultados = pd.DataFrame(data=resultados, columns=columns)
# 
#         print(resultados, end="\n")
# 
#         print("\nMejor conjunto de parámetros:")
#         print(model.best_params_, end="\n")
#         
#         ytrain_pred = model.predict(X.drop(["PID"], axis=1))
# 
#         # Encuentro la primera mejor configuración de parámetros:
#         if model.cv_results_['mean_test_score'].max() > mejormeanscore:
#             mejormeanscore = model.cv_results_['mean_test_score'].max()
#             mejorloss = loss
#             mejorpenalty = penalty
#             mejoreta0 = model.best_params_['eta0']
#             mejoralpha = model.best_params_['alpha']
#             predytrain = ytrain_pred
#             
#         
# # PARA LA MEJOR CONFIG ENCONTRADA:     
# print('\n\n================================================================================')
# 
# print('\nLos mejores parámetros encontrados son:\n   Función de costo: \"%s\"' % mejorloss,
#           '\n   Tipo de regularizador: \"%s\"' % mejorpenalty,
#          '\n   Tasa de entrenamiento', mejoreta0, '\n   Tasa de regularización', mejoralpha)
# 
# print('\nAccuracy para entrenamiento es: %.3f'% accuracy_score(y_train, predytrain))
# 
# print('\nReporte de clasificación para entrenamiento: \n\n', classification_report(y_train, predytrain))
# 
# plot_confusion_matrix(confusion_matrix(y_train, predytrain),classes=['0','1'],
#                       title="Matriz de confusión, Entrenamiento")
# 
# plt.show()
# 

### SVM

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [33]:
from sklearn.svm import SVC

In [34]:
# PRUEBO PRIMERO CON BÚSQUEDA MANUAL PARA VER CUÁNTO TARDA CORRER UN SOLO MODELO. 
# DEMORA MUCHO, NO HAGO EL GRID SEARCH. JOACO LO HIZO

# SupVectM = SVC(kernel='linear', max_iter=-1, random_state=42)
# SupVectM.fit(X.drop(["PID"], axis=1), y)
# y_pred= SupVectM.predict(X.drop(["PID"], axis=1))
# 
# print(accuracy_score(y, y_pred))
# results = results.append({'clf': SupVectM, 'best_acc': accuracy_score(y, y_pred)}, ignore_index=True)

In [35]:
# grid_param = {'C':(1,5,10), 'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
#               'shrinking':(True, False), 'class_weight':[None,'balanced']  }

# SupVectM = SVC(decision_function_shape='ovo', random_state=42)
# SupVectM_clf = GridSearchCV(SupVectM, param_grid=grid_param, scoring='accuracy', cv=5, iid=False)
# SupVectM_clf.fit(X.drop(["PID"], axis=1), y);
# best_svm_clf = SupVectM_clf.best_estimator_
# 
# print('Mejor accuracy', SupVectM_clf.best_score_)
# print(best_svm_clf)

# results = results.append({'clf': best_svm_clf, 'best_acc': SupVectM_clf.best_score_}, ignore_index=True)

### Regresión logística para multiclase

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
grid_param = {'solver':('sag', 'newton-cg'), 'C':(0.01, 0.1)}

reglog = LogisticRegression( multi_class='multinomial', random_state=42)
reglog_clf = GridSearchCV(reglog, grid_param, scoring='accuracy', cv=5, iid=False)
reglog_clf.fit(X.drop(["PID"], axis=1), y)
best_reglog_clf = reglog_clf.best_estimator_
print('Best Decision Tree accuracy: ', reglog_clf.best_score_)
print(best_reglog_clf)

results = results.append({'clf': best_reglog_clf, 'best_acc': reglog_clf.best_score_}, ignore_index=True)

C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\linear_model\sag.py:337: C

Best Decision Tree accuracy:  0.37006010781418197
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)


C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [38]:
pd.DataFrame(reglog_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,25.997862,0.307952,0.079308,9.381799e-02,0.01,sag,"{'C': 0.01, 'solver': 'sag'}",0.280925,0.273500,0.280246,0.284499,0.281797,0.280193,0.003645,3
1,108.257731,8.912460,0.030154,7.483159e-04,0.01,newton-cg,"{'C': 0.01, 'solver': 'newton-cg'}",0.359301,0.374114,0.358223,0.354442,0.356974,0.360611,0.006943,2
2,27.322841,3.251353,0.028955,6.143617e-07,0.1,sag,"{'C': 0.1, 'solver': 'sag'}",0.280925,0.273500,0.280246,0.284499,0.281797,0.280193,0.003645,3
3,130.323132,7.630409,0.029753,1.466934e-03,0.1,newton-cg,"{'C': 0.1, 'solver': 'newton-cg'}",0.374882,0.382617,0.364839,0.359641,0.368322,0.370060,0.007997,1


### KNN

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [39]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
# Búsqueda manual, no tiene muchos más parámetros
KNN = KNeighborsClassifier(n_neighbors=8)
KNN.fit(X.drop(["PID"], axis=1), y)
y_pred= KNN.predict(X.drop(["PID"], axis=1))

print(accuracy_score(y, y_pred))
results = results.append({'clf': KNN, 'best_acc': accuracy_score(y, y_pred)}, ignore_index=True) # MODIFICADO AL FINAL!!

0.5033075033075033


https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html

In [41]:
from sklearn.neighbors import NearestCentroid

In [42]:
grid_param = {'metric':('euclidean', 'manhattan'), 'shrink_threshold':(0.1,0.5)}

necent = NearestCentroid()
necent_clf = GridSearchCV(necent, grid_param, scoring='accuracy', cv=5, iid=False)
necent_clf.fit(X.drop(["PID"], axis=1), y)
best_necent_clf = necent_clf.best_estimator_
print('Best accuracy: ', necent_clf.best_score_)
print(best_necent_clf)

results = results.append({'clf': best_necent_clf, 'best_acc': necent_clf.best_score_}, ignore_index=True)

Best accuracy:  0.30703063723271595
NearestCentroid(metric='manhattan', shrink_threshold=0.5)


### Naive Bayes

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html

In [43]:
from sklearn.naive_bayes import BernoulliNB

In [44]:
naive = BernoulliNB(binarize=None, fit_prior=True)
naive.fit(X.drop(["PID"], axis=1), y)
y_pred= naive.predict(X.drop(["PID"], axis=1))

print(accuracy_score(y, y_pred))
results = results.append({'clf': naive, 'best_acc': accuracy_score(y, y_pred)}, ignore_index=True) 
# MUY MALO, NI VALE LA PENA MODIFICARLO!!

0.026932526932526932


C:\Users\Flor\Anaconda3\envs\diplodatos\lib\site-packages\sklearn\naive_bayes.py:955: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))


## Veo todos los modelos que se corrieron y cuál es el mejor según la accuracy

In [45]:
results

,clf,best_acc
0,"DecisionTreeClassifier(class_weight=None, crit...",0.351962
1,"DecisionTreeClassifier(class_weight=None, crit...",0.380077
2,"(DecisionTreeClassifier(class_weight=None, cri...",0.377621
3,"(DecisionTreeClassifier(class_weight=None, cri...",0.380267
4,"(DecisionTreeClassifier(class_weight=None, cri...",0.380079
5,"LogisticRegression(C=0.1, class_weight=None, d...",0.370060
6,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.503308
7,"NearestCentroid(metric='manhattan', shrink_thr...",0.307031
8,"BernoulliNB(alpha=1.0, binarize=None, class_pr...",0.026933


In [46]:
# printeo el mejor clasificador según mayor score.
print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

The best classifier so far is: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')


__Conclusión: parece ser que el knn da mejor resultado. Pero, recordemos que con GridSearchCV obtenemos el mejor modelo como aquel que da el máximo mean_test_score de accuracy, que es el que surge de hacer el promedio las accuracys obtenidas en cada partición del cross validation. Luego, para knn no se hizo cross validation sino que solo se tomó el accuracy del X total. Por ello no estoy tan segura de compararlos para decidir.__

__Como los modelos tardaron bastante en correr hago las correcciones abajo:__

### KNN con cross validation

In [47]:
from sklearn.model_selection import cross_validate

In [52]:
# Búsqueda manual, no tiene muchos más parámetros
KNN = KNeighborsClassifier(n_neighbors=8)
KNNCV = cross_validate(KNN, X.drop(["PID"], axis=1), y, scoring='accuracy', cv=5)
print('Mean_test_score para KNN con cross validation', KNNCV['test_score'].mean())

Mean_test_score para KNN con cross validation 0.335474788171386


In [70]:
results.iloc[6] = [KNN, KNNCV['test_score'].mean()]

__Naive bayes tambien debería corregirlo, pero como da muy malo su accuracy incluso sin CV, lo dejo como está porque no queda entre los mejores modelos de results.__

In [71]:
results

,clf,best_acc
0,"DecisionTreeClassifier(class_weight=None, crit...",0.351962
1,"DecisionTreeClassifier(class_weight=None, crit...",0.380077
2,"(DecisionTreeClassifier(class_weight=None, cri...",0.377621
3,"(DecisionTreeClassifier(class_weight=None, cri...",0.380267
4,"(DecisionTreeClassifier(class_weight=None, cri...",0.380079
5,"LogisticRegression(C=0.1, class_weight=None, d...",0.37006
6,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.335475
7,"NearestCentroid(metric='manhattan', shrink_thr...",0.307031
8,"BernoulliNB(alpha=1.0, binarize=None, class_pr...",0.0269325
9,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.335475


In [75]:
# printeo el mejor clasificador según mayor score.
print('The best classifier so far is: ')
print(results.loc[results['best_acc'].astype(float).idxmax()]['clf'])

The best classifier so far is: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=19, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=15,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [76]:
results['best_acc'].astype(float).idxmax()

3

## Mejores modelos subidos a kaggle y evaluados con el conjunto de test:

<br><br>
LogisticRegression(solver='newton-cg', multi_class='multinomial', C=0.1, max_iter = 200, random_state=42)

Public score: 0.33560
<br><br>


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,verbose=False)

Public score: 0.35071
<br><br> 


DT(class_weight=None, criterion='entropy', max_depth=7, max_features=None, max_leaf_nodes=None,            min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=200,            min_weight_fraction_leaf=0.0, presort=False, random_state=42, splitter='best')

Public score: 0.35298
<br><br>


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=19, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=15,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
                       
Public score: 0.34240